In [1]:
import pandas as pd
import numpy as np
import re
from unidecode import unidecode

pd.set_option('display.float_format', lambda x: '%.3f' % x)

In [2]:
transferencias = pd.read_csv('../../data/guardian_transfer_data/guardian_transfer_data.csv')

#### Procesamiento Transferencias

In [3]:
transferencias = transferencias[(transferencias['transfer_year'] <= 2019) &  
                                (transferencias['price_in_pounds'].isna()==False) & 
                                (transferencias['transfer_type'] == 'fee')]

#transferencias['player_name'] = transferencias['player_name'].apply(lambda x: re.sub(r'[^\w\s]', '', x).lower())

transferencias['player_name'] = transferencias['player_name'].apply(lambda x: unidecode(re.sub(r'[^\w\s]', '', x)).lower())


"""
transferencias_train = transferencias[(transferencias['transfer_year'] <= 2018) &  
                                (transferencias['price_in_pounds'].isna()==False) & 
                                (transferencias['transfer_type'] == 'fee')]

transferencias_test = transferencias[(transferencias['transfer_year'] == 2019) &  
                                (transferencias['price_in_pounds'].isna()==False) & 
                                (transferencias['transfer_type'] == 'fee')]
"""

mean_transfer_team = pd.DataFrame(transferencias.groupby('new_club')['price_in_pounds'].mean().reset_index())

mean_transfer_team['new_club'] = mean_transfer_team['new_club'].astype(str).apply(lambda x: unidecode(re.sub(r'[^\w\s]', '', x)).lower())


#### Procesamiento VAEP

In [4]:
vaep = pd.read_csv('../../data/vaep/xgboost_vaep.csv')
vaep['player_name'] = vaep['first_name'] + ' ' + vaep['last_name']
vaep['player_name'] = vaep['player_name'].astype(str)
vaep['player_name'] = vaep['player_name'].apply(lambda x: unidecode(re.sub(r'[^\w\s]', '', x)).lower())

In [5]:
vaep_grouped = vaep.groupby('player_name')[['offensive_value','defensive_value','vaep_value','scores','concedes']].mean().reset_index()

In [6]:
transferencias_vaep = vaep_grouped.merge(transferencias, on = 'player_name', how = 'inner')

In [7]:
transferencias_vaep

,player_name,offensive_value,defensive_value,vaep_value,scores,concedes,transfer_id,transfer_year,window_day,window_open,...,transfer_status,transfer_type,primary_player_position,price_in_pounds,previous_club,new_club,previous_league,new_league,big_deal,flow_id
0,aaron mooy,0.004,0.000,0.004,0.015,0.004,1,2017,-1,2017-07-01,...,done deal,fee,NaN,8000000.000,Manchester City,Huddersfield Town,Premier League,Premier League,No,7
1,aaron wanbissaka,0.003,0.000,0.003,0.012,0.006,1,2019,44,2019-05-16,...,done deal,fee,D,45000000.000,Crystal Palace,Manchester United,Premier League,Premier League,Yes,80
2,abdelhamid sabiri,-0.000,0.000,-0.000,0.014,0.005,1,2017,53,2017-07-01,...,done deal,fee,NaN,1000000.000,FC Nürnberg,Huddersfield Town,2. Bundesliga,Premier League,No,3
3,abdou diallo,0.003,-0.001,0.002,0.011,0.007,1,2019,61,2019-05-16,...,done deal,fee,D,28900000.000,Borussia Dortmund,Paris Saint-Germain,1. Bundesliga,Ligue 1,Yes,8
4,abdou diallo,0.003,-0.001,0.002,0.011,0.007,1,2018,40,2018-05-17,...,done deal,fee,D,22750000.000,Mainz 05,Borussia Dortmund,1. Bundesliga,1. Bundesliga,NaN,65
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
575,youssef ennesyri,0.003,0.000,0.003,0.025,0.005,1,2018,93,2018-05-17,...,done deal,fee,F,5400000.000,Málaga,Leganés,Spanish Segunda,La Liga,NaN,35
576,youssouf sabaly,0.003,0.000,0.003,0.012,0.004,1,2017,-1,2017-07-01,...,done deal,fee,NaN,3500000.000,Paris Saint-Germain,Bordeaux,Ligue 1,Ligue 1,No,19
577,yuya osako,0.005,0.001,0.006,0.026,0.004,1,2018,14,2018-05-17,...,done deal,fee,F,5130000.000,Cologne,Werder Bremen,Bundesliga 2.,1. Bundesliga,NaN,5
578,yves bissouma,0.003,0.000,0.003,0.014,0.005,1,2018,61,2018-05-17,...,done deal,fee,M,15000000.000,Lille,Brighton,Ligue 1,Premier League,YES,16


In [8]:
transferencias_vaep_train = transferencias_vaep[transferencias_vaep['transfer_year'] <= 2018]
transferencias_vaep_test = transferencias_vaep[transferencias_vaep['transfer_year'] == 2019]

In [9]:
transferencias_vaep_train[['player_name','window_day','window_open','timestamp']].sort_values('window_day',ascending = True)

,player_name,window_day,window_open,timestamp
574,youri tielemans,-38,2017-07-01,2017-05-24
360,matthias ostrzolek,-38,2017-07-01,2017-05-24
81,benoit costil,-37,2017-07-01,2017-05-25
146,enes unal,-12,2017-07-01,2017-06-19
448,rani khedira,-12,2017-07-01,2017-06-19
...,...,...,...,...
203,guessouma fofana,104,2018-05-17,2018-08-29
388,nacer chadli,105,2018-05-17,2018-08-30
398,nemanja radonjic,105,2018-05-17,2018-08-30
298,lebo mothiba,106,2018-05-17,2018-08-31


In [10]:
transferencias_vaep_test

,player_name,offensive_value,defensive_value,vaep_value,scores,concedes,transfer_id,transfer_year,window_day,window_open,...,transfer_status,transfer_type,primary_player_position,price_in_pounds,previous_club,new_club,previous_league,new_league,big_deal,flow_id
1,aaron wanbissaka,0.003,0.000,0.003,0.012,0.006,1,2019,44,2019-05-16,...,done deal,fee,D,45000000.000,Crystal Palace,Manchester United,Premier League,Premier League,Yes,80
3,abdou diallo,0.003,-0.001,0.002,0.011,0.007,1,2019,61,2019-05-16,...,done deal,fee,D,28900000.000,Borussia Dortmund,Paris Saint-Germain,1. Bundesliga,Ligue 1,Yes,8
7,adam nagy,0.003,0.000,0.003,0.013,0.005,1,2019,84,2019-05-16,...,done deal,fee,M,2250000.000,Bologna,Bristol City,Serie A,Championship,NaN,11
12,adem ljajic,0.006,-0.000,0.006,0.025,0.004,1,2019,33,2019-05-16,...,done deal,fee,M,6000000.000,Torino,Besiktas,Serie A,Turkish Super Lig,NaN,10
13,ademola lookman,0.013,-0.000,0.013,0.037,0.005,1,2019,70,2019-05-16,...,done deal,fee,W,22500000.000,Everton,RB Leipzig,Premier League,1. Bundesliga,Yes,30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
560,wesley said,0.011,-0.001,0.010,0.031,0.005,1,2019,79,2019-05-16,...,done deal,fee,F,7200000.000,Dijon,Toulouse,Ligue 1,Ligue 1,NaN,134
565,wissam ben yedder,0.009,-0.000,0.008,0.041,0.004,1,2019,90,2019-05-16,...,done deal,fee,F,37100000.000,Sevilla,Monaco,La Liga,Ligue 1,Yes,26
568,yannis salibur,0.003,-0.000,0.003,0.024,0.003,1,2019,98,2019-05-16,...,done deal,fee,W,1800000.000,Guingamp,Real Mallorca,Ligue 2,La Liga,NaN,6
569,yevhen konoplyanka,0.004,-0.000,0.003,0.025,0.004,1,2019,109,2019-05-16,...,done deal,fee,W,5400000.000,Schalke 04,Shakhtar Donetsk,La Liga,Ukrainian Premier League,NaN,1


#### Procesamiento FBREF

In [11]:
#fbref = pd.read_csv('../../data/datos_scrappeados/')

In [12]:
#revision_jugadores = vaep_grouped.merge(transferencias, on = 'player_name', how = 'outer')

In [13]:
#revision_jugadores

## Primer modelo

In [14]:
import xgboost as xgb
from sklearn.datasets import load_boston
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import OneHotEncoder

In [15]:
X = ['offensive_value', 'defensive_value', 'vaep_value',
       'scores', 'concedes', 'transfer_year', 'window_day',
       'primary_player_position', 'previous_club',
       'new_club', 'previous_league', 'new_league','big_deal','price_in_pounds']

Y = 'price_in_pounds'

In [16]:
transferencias_vaep = transferencias_vaep[X]

In [17]:
#Encode non-numeric data

# AUX function to put categorical data into one-hot encoding format
def to_categorical(var):
    enc = OneHotEncoder(sparse=False)
    var = enc.fit_transform(var)
    #print("== categories:", enc.categories_, "==")
    return var, [item for sublist in enc.categories_ for item in sublist]

In [18]:
new_club, new_club_map = to_categorical(transferencias_vaep[['new_club']])
previous_club, previous_club_map = to_categorical(transferencias_vaep[['previous_club']])
previous_league, previous_league_map = to_categorical(transferencias_vaep[['previous_league']])
new_league, new_league_map = to_categorical(transferencias_vaep[['new_league']])
primary_player_position, primary_player_position_map = to_categorical(transferencias_vaep[['primary_player_position']])
transferencias_vaep['big_deal'].replace(['No', 'Yes', float("nan"), 'YES'],[0,1,0,1],inplace = True)

/var/folders/c7/yrk77vqj3jj31wf8sfvg8tyw0000gn/T/ipykernel_20944/2188447406.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  transferencias_vaep['big_deal'].replace(['No', 'Yes', float("nan"), 'YES'],[0,1,0,1],inplace = True)


In [19]:
new_club_df = pd.DataFrame(new_club,columns = new_club_map)
previous_club_df = pd.DataFrame(previous_club,columns = previous_club_map)
previous_league_df = pd.DataFrame(previous_league,columns = previous_league_map)
new_league_df = pd.DataFrame(new_league,columns = new_league_map)
primary_player_position_df = pd.DataFrame(primary_player_position,columns = primary_player_position_map)

In [20]:
transferencias_vaep.drop(columns = ['new_league','previous_league', 'new_club', 'previous_club','primary_player_position'],inplace = True)

/var/folders/c7/yrk77vqj3jj31wf8sfvg8tyw0000gn/T/ipykernel_20944/1330882033.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  transferencias_vaep.drop(columns = ['new_league','previous_league', 'new_club', 'previous_club','primary_player_position'],inplace = True)


In [21]:
transferencias_vaep_encoded = pd.concat([transferencias_vaep,new_club_df,previous_club_df,
                                         previous_league_df,new_league_df,primary_player_position_df],axis = 1)

In [22]:
transferencias_vaep_encoded.to_csv('../../data/modelo/transferencias_vaep_encoded.csv')

In [23]:
transferencias_vaep_train = transferencias_vaep_encoded[transferencias_vaep_encoded['transfer_year'] <= 2018]
transferencias_vaep_test = transferencias_vaep_encoded[transferencias_vaep_encoded['transfer_year'] == 2019]

In [24]:
#Split dataframe in train and test
X_train, y_train = transferencias_vaep_train,transferencias_vaep_train[Y]
X_test, y_test = transferencias_vaep_test, transferencias_vaep_test[Y]

X_train.drop(columns = [Y],inplace = True)
X_test.drop(columns = [Y],inplace = True)

/var/folders/c7/yrk77vqj3jj31wf8sfvg8tyw0000gn/T/ipykernel_20944/76074295.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train.drop(columns = [Y],inplace = True)
/var/folders/c7/yrk77vqj3jj31wf8sfvg8tyw0000gn/T/ipykernel_20944/76074295.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test.drop(columns = [Y],inplace = True)


In [25]:
X_train = np.array(X_train)
X_test = np.array(X_test)

y_train = np.array(y_train)
y_test = np.array(y_test)

In [26]:
# Define the XGBoost model
xgb_model = xgb.XGBRegressor(objective='reg:squarederror', colsample_bytree=0.3, learning_rate=0.1,
                max_depth=20, alpha=10, n_estimators=100)

# Train the model on the training data
xgb_model.fit(X_train, y_train)

# Make predictions on the test data
y_pred = xgb_model.predict(X_test)

# Evaluate the model performance using mean squared error
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error: ", mse)

Mean Squared Error:  143905558194445.34
